In [9]:
# ============================================
# Brain Tumor Detection with 2 Loss Functions
# Binary Crossentropy & Categorical Crossentropy
# ============================================

import os
import zipfile
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models


In [16]:
# ==============================
# 1. Extract dataset
# ==============================
from google.colab import files

uploaded = files.upload()

zip_path = "/content/project4_data.zip"   # uploaded path in Colab
extract_dir = "/content/data"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("Data extracted to:", extract_dir)

Data extracted to: /content/data


In [17]:
# ==============================
# 2. Preprocess Images
# ==============================
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)


In [18]:
# Binary classification generator
train_gen_binary = datagen.flow_from_directory(
    extract_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

val_gen_binary = datagen.flow_from_directory(
    extract_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

Found 3089 images belonging to 4 classes.
Found 772 images belonging to 4 classes.


In [19]:
# Multi-class classification generator
train_gen_cat = datagen.flow_from_directory(
    extract_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_gen_cat = datagen.flow_from_directory(
    extract_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

Found 3089 images belonging to 4 classes.
Found 772 images belonging to 4 classes.


In [20]:
# ==============================
# 3. Define CNN Models
# ==============================
def build_cnn_binary():
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=(*IMG_SIZE, 3)),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(128, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid')  # binary
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_cnn_categorical(num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=(*IMG_SIZE, 3)),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(128, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation='softmax')  # multi-class
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [21]:
# ==============================
# 4. Train Models
# ==============================
print("\n=== Training Binary Crossentropy Model ===")
binary_model = build_cnn_binary()
history_bin = binary_model.fit(
    train_gen_binary,
    validation_data=val_gen_binary,
    epochs=10
)

print("\n=== Training Categorical Crossentropy Model ===")
num_classes = train_gen_cat.num_classes
cat_model = build_cnn_categorical(num_classes)
history_cat = cat_model.fit(
    train_gen_cat,
    validation_data=val_gen_cat,
    epochs=10
)


=== Training Binary Crossentropy Model ===


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 118s 1s/step - accuracy: 0.3709 - loss: -3275781.7500 - val_accuracy: 0.3886 - val_loss: -146513424.0000
Epoch 2/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 106s 1s/step - accuracy: 0.3843 - loss: -1255806208.0000 - val_accuracy: 0.3886 - val_loss: -12652858368.0000
Epoch 3/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 105s 1s/step - accuracy: 0.3872 - loss: -33601888256.0000 - val_accuracy: 0.3886 - val_loss: -151187193856.0000
Epoch 4/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 103s 1s/step - accuracy: 0.3841 - loss: -294268469248.0000 - val_accuracy: 0.3886 - val_loss: -795353939968.0000
Epoch 5/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 101s 1s/step - accuracy: 0.3966 - loss: -1235894927360.0000 - val_accuracy: 0.3886 - val_loss: -2753568964608.0000
Epoch 6/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 106s 1s/step - accuracy: 0.3817 - loss: -4207538601984.0000 - val_accuracy: 0.3886 - val_loss: -7281498914816.0000
Epoch 7/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 105s 1s/step - accuracy: 0.3807 - loss: -10874309836800.0000 - v

In [22]:
# ==============================
# 5. Evaluate Models
# ==============================
def evaluate_model(model, val_gen, model_type="Binary"):
    val_gen.reset()
    preds = model.predict(val_gen)

    if model_type == "Binary":
        y_pred = (preds > 0.5).astype(int).ravel()
        y_true = val_gen.classes
    else:
        y_pred = np.argmax(preds, axis=1)
        y_true = val_gen.classes

    print(f"\n=== {model_type} Model Report ===")
    print(classification_report(y_true, y_pred, target_names=list(val_gen.class_indices.keys())))
    print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))

In [23]:
# Evaluate Binary
evaluate_model(binary_model, val_gen_binary, "Binary")

25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 278ms/step

=== Binary Model Report ===
                 precision    recall  f1-score   support

Br35H-Mask-RCNN       0.00      0.00      0.00       160
             no       0.39      1.00      0.56       300
           pred       0.00      0.00      0.00        12
            yes       0.00      0.00      0.00       300

       accuracy                           0.39       772
      macro avg       0.10      0.25      0.14       772
   weighted avg       0.15      0.39      0.22       772

Confusion Matrix:
 [[  0 160   0   0]
 [  0 300   0   0]
 [  0  12   0   0]
 [  0 300   0   0]]


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [24]:
# Evaluate Categorical
evaluate_model(cat_model, val_gen_cat, "Categorical")

25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 316ms/step

=== Categorical Model Report ===
                 precision    recall  f1-score   support

Br35H-Mask-RCNN       0.86      0.04      0.07       160
             no       0.39      0.39      0.39       300
           pred       0.00      0.00      0.00        12
            yes       0.38      0.58      0.46       300

       accuracy                           0.39       772
      macro avg       0.41      0.25      0.23       772
   weighted avg       0.48      0.39      0.35       772

Confusion Matrix:
 [[  6  56   1  97]
 [  0 118   0 182]
 [  0   5   0   7]
 [  1 124   0 175]]
